In [3]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import sys
from osgeo import gdal, ogr, gdal_array, osr
import shapely.geometry
import multiprocessing as mp
import datetime
import tempfile
#from hhnk_research_tools import raster_functions as rf
#from hhnk_research_tools import Raster
import hhnk_research_tools as hrt
import rasterio 

#setting paths
config = {
    "landuse": r"E:\01.basisgegevens\rasters\landgebruik\landuse2020_tiles\combined_rasters.vrt",
    "damagecurve": " ",
    "dem": r"E:\01.basisgegevens\rasters\DEM\DEM_AHN4_int.vrt",
    "peilgebieden": r"E:\01.basisgegevens\hhnk_schadeschatter\01_data\peilgebieden_Marken.shp",
    "output_folder": r"G:\02_Werkplaatsen\06_HYD\Projecten\HKC23013 Schadecurve 2.0\08. schadeschatter_v2\OUTPUT"
}

pgb_shp = gpd.read_file(config['peilgebieden']) #CHANGE
dem = hrt.Raster(config['dem'])
output_folder = config['output_folder']

ModuleNotFoundError: No module named 'threedi_api_client'

In [ ]:
#peilgebieden shapefile to raster
meta=hrt.create_meta_from_gdf(gdf=pgb_shp, res=dem.metadata['pixel_width'])
hrt.gdf_to_raster(gdf=pgb_shp, value_field='peil_id', raster_out=str(output_folder + '\combined1.tiff'),
    nodata=0, metadata=meta, epsg=28992, driver='GTiff')


In [ ]:
def define_lowest_point(rasterfile):
    rasterfile = rasterfile 
    percentiles = [5] 
    rasterArray = gdal_array.LoadFile(rasterfile) #Read raster as numpy array
    rasterArray = np.where(rasterArray==-9999, np.nan, rasterArray)

    for p in percentiles:
        lowest_1 = np.nanpercentile(rasterArray,p)
        print(lowest_1)
    return lowest_1


In [147]:
for index, row in pgb_shp.iterrows():

    raster_pgb = hrt.Raster(os.path.join(output_folder, f'pgb_{row.peil_id}.tif'))
    data = row[['peil_id', 'streefpeil', 'geometry']].to_dict()
    sub_df = gpd.GeoDataFrame(data=[data])
    meta= hrt.create_meta_from_gdf(gdf=sub_df, res=dem.metadata['pixel_width'])

    hrt.gdf_to_raster(gdf=sub_df,
                value_field='peil_id',
                raster_out= raster_pgb,
                nodata=-9999,
                metadata=meta,
                read_array=False,
                overwrite=False)
    break

In [142]:
blocks_df = raster_pgb.generate_blocks_geometry()
for index, row in blocks_df.iterrows():
    

In [146]:
blocks_df[['ix','iy','geometry']].to_file("blocks.gpkg")